In [85]:
%matplotlib inline
%config InlineBackend.figure_format="retina"
import numpy as np
import pandas as pd
from keras.models import load_model
import torch
import os
from spotify import SpotifyAPI
from dotenv import load_dotenv

In [ ]:
#Commented runs use these to run other notebooks (genius.ipynb not included because the lyrics are saved)
# %processing.ipynb
# %run bert.ipynb

In [62]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
%run mrs.ipynb

# of rows of Song Data: 50683
# of unique songs: 50683
# of rows of User Listening Data: 806745
# of unique users: 25343
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50683 entries, 0 to 50682
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   track_id          50683 non-null  object 
 1   name              50683 non-null  object 
 2   artist            50683 non-null  object 
 3   spotify_id        50683 non-null  object 
 4   tags              49556 non-null  object 
 5   danceability      50683 non-null  float64
 6   energy            50683 non-null  float64
 7   loudness          50683 non-null  float64
 8   mode              50683 non-null  int64  
 9   speechiness       50683 non-null  float64
 10  acousticness      50683 non-null  float64
 11  instrumentalness  50683 non-null  float64
 12  liveness          50683 non-null  float64
 13  valence           50683 non-null  float64
 14  tempo        

C:\Users\cam-m\AppData\Local\Temp\ipykernel_14596\1274972821.py:5: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\cam-m\AppData\Local\Temp\ipykernel_14596\1274972821.py:8: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



69
[8 9 3 7 4 5 2 1 6 0]


In [98]:
#Note that the bert model and the mrs model need to run before this

x_test = np.load("misc/x_test.npy")
y_test = np.load("misc/y_test.npy")
model = load_model("misc/mae_optimized_model.keras")
lyrics_embeddings_csv = pd.read_csv('misc/lyrics_embeddings.csv')
lyrics_embeddings_3d_csv = pd.read_csv('misc/lyrics_embeddings_3d.csv')
distance_frame = pd.read_csv('misc/distance_frame.csv')
spotify_input = np.load("misc/spotify_data.npy")
compare_spotify = np.load("misc/compare_spotify.npy")
spotify_input_names = np.load("misc/our_playlist_names.npy")
dataset = pd.read_csv("misc/processed_music_info_extended.csv")
distance_frame = distance_frame.drop(['Unnamed: 0'], axis=1)

In [64]:
lyrics_embeddings, lyrics_embeddings_3d = get_embeddings(lyrics_embeddings_csv, lyrics_embeddings_3d_csv)

C:\Users\cam-m\AppData\Local\Temp\ipykernel_14596\1274972821.py:5: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\cam-m\AppData\Local\Temp\ipykernel_14596\1274972821.py:8: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [99]:
distance_frame.head(20)

,track_id,name,spotify_id,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence
0,TRIOREW128F424EAF0,Mr. Brightside,09ZQ5TmUG8TSL56n0knqrj,0.355,0.918,-4.360,0.0746,0.001190,0.000000,0.0971,0.240
1,TRRIVDJ128F429B0E8,Wonderwall,06UfBBDISthj1ZJAtX4xjj,0.409,0.892,-4.373,0.0336,0.000807,0.000000,0.2070,0.651
2,TROUVHL128F426C441,Come as You Are,0keNu0t0tqsWtExGM3nT1D,0.508,0.826,-5.783,0.0400,0.000175,0.000459,0.0878,0.543
3,TRUEIND128F93038C4,Take Me Out,0ancVQ9wEcHVd0RrGICTE4,0.279,0.664,-8.851,0.0371,0.000389,0.000655,0.1330,0.490
4,TRLNZBD128F935E4D8,Creep,01QoK9DA7VTeTSE3MNzp4I,0.515,0.430,-9.935,0.0369,0.010200,0.000141,0.1290,0.104
5,TRUMISQ128F9340BEE,Somebody Told Me,0FNmIQ7u45Lhdn6RHhSLix,0.508,0.979,-4.289,0.0847,0.000087,0.000643,0.0641,0.704
6,TRVCCWR128F9304A30,Viva la Vida,08A1lZeyLMWH58DT6aYjnC,0.588,0.806,-7.903,0.1050,0.153000,0.000000,0.0634,0.520
7,TRXOGZT128F424AD74,Karma Police,01puceOqImrzSfKDAcd1Ia,0.360,0.505,-9.129,0.0260,0.062600,0.000092,0.1720,0.317
8,TRMZXEW128F9341FD5,The Scientist,0GSSsT9szp0rJkBrYkzy6s,0.566,0.429,-7.826,0.0242,0.715000,0.000014,0.1200,0.173
9,TRUJIIV12903CA8848,Clocks,0BCPKOYdS2jbQ8iyB56Zns,0.577,0.749,-7.215,0.0279,0.599000,0.011500,0.1830,0.255


In [74]:
POTENTIAL_N = 50

predicted = predict_sample(x_test[2], model)
print(predicted)
recommendation = pipeline_helper(distance_frame, predicted, POTENTIAL_N, lyrics_embeddings, dataset)  
recommendation


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
[ 5.5469018e-01  6.4444405e-01 -7.1021342e+00  3.6774069e-02
  1.4361450e-01  4.7395937e-04  1.3090470e-01  5.0706518e-01]
                                                   id  distance
Die Born                           TRGTKSR128F4222CD7  0.221414
Shout Out Loud                     TRIJOOM128EF344100  0.235216
You've Got To Stand For Something  TRABSQC128F428AD3D  0.243626
Sweetest Thing                     TRALVXD128F1495205  0.247865
Cold as Ice                        TRUUEBK128F428F425  0.253281
Please Don't Die                   TRTWVWU128F145267B  0.260793
Gray Death                         TROBZDH12903CB5DBA  0.280711
Angels or Devils                   TRXCIRY128F92E81CC  0.288603
Don't Speak                        TRGAIDZ128F92EF7A1  0.294355
Most Of The Time                   TRGDCAT12903D05D17  0.306716
Birmingham                         TRIBUKY128EF35F68E  0.316152
I Breathe                          TRXEHWF128F9346707  0.326935
Happily

,name,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence
103,Don't Speak,0.522,0.707,-7.113,0.0354,0.24000,0.000020,0.1560,0.572
1434,It Must Have Been Love,0.564,0.658,-7.102,0.0251,0.02840,0.002830,0.1700,0.663
3303,Sunday Sun,0.521,0.722,-7.140,0.0259,0.22200,0.007830,0.1380,0.339
9259,Shout Out Loud,0.564,0.658,-7.106,0.0265,0.05560,0.011700,0.1070,0.432
27233,Soulfood,0.572,0.671,-6.914,0.0616,0.09200,0.000000,0.1410,0.552
32470,Falling Through Your Clothes,0.548,0.694,-6.989,0.0283,0.23300,0.000031,0.1870,0.538
34565,Galaxies,0.582,0.767,-7.034,0.0298,0.08990,0.000000,0.1330,0.388
39380,Loyalty Song,0.488,0.672,-7.035,0.0267,0.00102,0.000224,0.0969,0.449
41146,I Breathe,0.525,0.678,-7.195,0.0329,0.04130,0.000000,0.1790,0.491
41195,Best Friends,0.606,0.474,-7.074,0.0263,0.05740,0.004340,0.0790,0.511


In [76]:
christina_songs = [[0.654,0.924,-6.28,0.0406,0.00556,0.144,0.331,0.718], [0.554,0.49,-8.046,0.0457,0.0168,0.000534,0.136,0.513], [0.415,0.867,-5.497,0.0387,5.69e-05,0.000192,0.0883,0.36], [0.597,0.812,-12.062,0.0315,0.0219,0.00106,0.102,0.823], [0.517,0.206,-12.233,0.0421,0.924,0.0,0.118,0.489]]
# song1 = np.asarray([0.654,0.924,-6.28,0,0.0406,0.00556,0.144,0.331,0.718])
# song2 = np.asarray([0.554,0.49,-8.046,0.0457,0.0168,0.000534,0.136,0.513])
# song3 = np.asarray([0.415,0.867,-5.497,0.0387,5.69e-05,0.000192,0.0883,0.36])
# song4 = np.asarray([0.597,0.812,-12.062,0.0315,0.0219,0.00106,0.102,0.823])
# song5 = np.asarray([0.517,0.206,-12.233,0.0421,0.924,0.0,0.118,0.489])
christina_songs_array = [np.array(sublist) for sublist in christina_songs]
christina_songs_array = np.array(christina_songs_array)

# print(type(christina_songs[0][0]))
c_predicted = predict_sample(christina_songs, model)
christina_recs = pipeline_helper(distance_frame, c_predicted, POTENTIAL_N, lyrics_embeddings, dataset) 
christina_recs

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
                                        id  distance
Black Dahlia            TRJSGIY12903CB1355  0.197580
Wildest Dreams          TRRJEHJ12903D063A2  0.236814
Who's Crying Now        TRWXJYH12903CA885C  0.238956
2 Wicky                 TRPHDLW12903CE840F  0.240267
Sick Boy                TRBNMWP128F4255849  0.257511
A Murder Of One         TRYCMNE128F42503BF  0.271846
Relention               TRPYWQA128EF348613  0.275246
Great Indoors           TRGYEZE128F92F52B2  0.295294
Insekter                TRULNKK128F42A9423  0.310660
Different Now           TRXTNBU128F4233BA3  0.321936
Verloren                TRRRSSX128F42BCE2A  0.322054
Breakeven               TRASGVG128F92E8E0F  0.325535
Elegance                TRRMIOV12903CC65C5  0.336674
Soma                    TRHBABG128F4281C8E  0.339176
StreetLove              TRIYBSB128F14B0259  0.340280
Nicotine Stain          TRHAKCB128E0793643  0.341432
Electricity             TRLLPCM128F93362F9  0.344608
Dark Hal

,name,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence
3269,Electricity,0.529,0.771,-7.403,0.0645,0.089900,0.046300,0.1970,0.421
12730,Shooting Star,0.517,0.798,-7.562,0.0350,0.019900,0.000364,0.2210,0.505
24657,A Murder Of One,0.553,0.615,-7.433,0.0378,0.119000,0.002810,0.1060,0.425
26560,Relention,0.556,0.647,-7.479,0.0322,0.005210,0.011300,0.1320,0.389
29272,Fallen Icons,0.502,0.714,-7.451,0.0260,0.262000,0.015300,0.1310,0.523
29821,You To Thank,0.363,0.678,-7.492,0.0390,0.089200,0.006190,0.1210,0.267
31739,Different Now,0.648,0.681,-7.470,0.0295,0.103000,0.001020,0.1030,0.364
32842,Action Woman,0.326,0.674,-7.505,0.0376,0.067900,0.000059,0.1700,0.495
37068,Better Days Will Come,0.543,0.777,-7.426,0.0277,0.000747,0.007450,0.1450,0.558
39372,If I Found Out,0.647,0.668,-7.503,0.0256,0.046300,0.000048,0.0787,0.595


### Spotify API Evaluation
Get recommendations from Spotify by passing in minimum and maximum feature values, and compare the songs to our recommended playlist.

In [92]:
from spotify import SpotifyAPI
from dotenv import load_dotenv
# Get the input playlist
# Traverse along each song in playlist and preserve min max of feature values and 
# Store in list that gets passed to parameters for Spotify APIcommendations

# input_playlist = og_songs
# features = dict()
# feature_names = ["danceability","energy","loudness","speechiness","acousticness","instrumentalness","liveness", "valence"]

# for feature in feature_names:
#     features["min_" + feature] = input_playlist[feature].min()
#     features["max_" + feature] = input_playlist[feature].max()

# features["seed_genres"] = input_playlist['tags'].iloc[0].split(',')[:3]
# features["seed_genres"] = ','.join(features["seed_genres"])
# features

In [95]:
print(compare_spotify[0])
print(spotify_input[0])

[[ 4.9800e-01  2.6400e-01 -2.0461e+01  4.1600e-02  4.3800e-01  4.8400e-01
   1.2400e-01  9.1800e-01]
 [ 6.2700e-01  4.6800e-01 -1.4815e+01  6.7700e-02  1.5500e-01  2.8100e-02
   9.6300e-02  4.1300e-01]
 [ 4.8900e-01  4.7500e-01 -7.8540e+00  2.4200e-02  1.7100e-01  2.6400e-02
   2.0900e-01  4.7400e-01]
 [ 3.1500e-01  5.7900e-01 -6.8260e+00  3.0900e-02  9.1100e-03  7.8900e-03
   1.5000e-01  4.0400e-01]
 [ 5.2300e-01  5.5000e-01 -7.1960e+00  2.7000e-02  5.2700e-01  7.4400e-01
   3.6700e-01  1.8900e-01]]
['0BYHHSxMS1zDQp85t4sxL0' '0d8roVOosN85HzEK0Oft3L'
 '1u2g7up9g5p7eXeuNyAXjt' '27ezR6SR0Q2YlpLXiMKP4y'
 '1XmIOdIP1Bnbd7ALE0dIi8']


In [96]:
print(spotify_input_names[0])

["I'm Still Your Fag" 'The Big Fight' 'In Our Bedroom After the War'
 'Skinny Boy' 'What the Snowman Learned About Love']


In [100]:
predicted = predict_sample(compare_spotify[0], model)
print(predicted)
our_recs = pipeline_helper(distance_frame, predicted, POTENTIAL_N, lyrics_embeddings, dataset) 
our_recs

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step
[ 0.52436244  0.6479608  -8.26962     0.04006468  0.12437931  0.01548875
  0.12973773  0.42364514]
                                                            id  distance
Dark Paradise                               TRQOIAM128F4255677  0.179193
Fireflies                                   TRILGSU128F424E631  0.236641
I Get My Beat                               TRBXCOJ128E07820C1  0.259874
When Mermaids Cry                           TRFZMEN128E0792E40  0.290782
I Wonder                                    TRLJQXJ12903D08ECB  0.293804
Sweetwater                                  TRBNQWH128F421FBFE  0.312098
If It Makes You Happy                       TRIZRZS128E078E1F8  0.312933
Little Black Sandals                        TRXUENF128F423D477  0.314328
You & A Promise                             TRJFSED128F42A0BCA  0.314345
We End Up Together                          TRLOBIT12903CB9FF8  0.317334
Why Do I Feel So Sad                        TRDFWVZ128F4284

,name,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence
15270,Abuse Me,0.470,0.676,-8.222,0.0307,0.273000,0.000026,0.1120,0.372
24068,Today Is A Good Day,0.499,0.501,-8.368,0.0273,0.115000,0.000148,0.0950,0.454
25586,So It Goes,0.382,0.647,-8.239,0.0343,0.001520,0.000096,0.0883,0.361
25760,Yama Yama,0.696,0.709,-8.274,0.0586,0.110000,0.000012,0.1850,0.336
29811,Stupid,0.543,0.486,-8.383,0.0302,0.147000,0.000003,0.0714,0.462
31147,Fingertips,0.565,0.599,-8.248,0.0403,0.228000,0.060800,0.1200,0.336
33365,Drunken Angel,0.472,0.560,-8.119,0.0357,0.085700,0.000000,0.1590,0.395
35575,Sweetwater,0.504,0.628,-8.304,0.0231,0.000821,0.000000,0.1140,0.358
38374,You & A Promise,0.562,0.684,-8.182,0.0285,0.065600,0.032100,0.1110,0.471
41101,Blåjeans,0.570,0.605,-8.162,0.0247,0.000557,0.001750,0.0913,0.424


In [101]:
env_path = os.path.join('misc', '.env')
load_dotenv(dotenv_path=env_path)
SPOTIFY_CREDS = [os.getenv('SPOTIFY_CLIENT_ID'), 
                os.getenv('SPOTIFY_CLIENT_SECRET')]

if not all(SPOTIFY_CREDS):
        print("Please set the SPOTIFY_CLIENT_ID and SPOTIFY_CLIENT_SECRET environment variables")
        exit(1)

spotify = SpotifyAPI(*SPOTIFY_CREDS)
SPOTIFY_CREDS

['1fa1ca9358074af48f6230e7b13c0737', '880a92c5194f496b929d0d4d96ec3a59']

In [102]:
print(spotify_input[0])

['0BYHHSxMS1zDQp85t4sxL0' '0d8roVOosN85HzEK0Oft3L'
 '1u2g7up9g5p7eXeuNyAXjt' '27ezR6SR0Q2YlpLXiMKP4y'
 '1XmIOdIP1Bnbd7ALE0dIi8']


In [103]:
recs_from_spotify = spotify.get_recommendations(seed_tracks=list(spotify_input[0]), limit=10)

In [104]:
# SPOTIFY's PREDICTED RECOMMENDATIONS
recs_from_spotify = recs_from_spotify.loc[:, ['name', 'danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness',
                                'liveness', 'valence']]
recs_from_spotify

,name,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence
0,Bittersweet Melodies,0.500,0.3990,-8.715,0.0287,0.83900,0.00013,0.0962,0.343
1,I've Got Your Fire,0.584,0.7460,-7.137,0.0341,0.63900,0.54800,0.1400,0.497
2,Look Up - Live at Morning Becomes Eclectic / 2007,0.653,0.0174,-34.895,0.0457,0.62600,0.00000,0.0855,0.294
3,Tugboat,0.285,0.8900,-8.143,0.0405,0.15300,0.49200,0.1130,0.323
4,Mistress,0.663,0.5420,-8.915,0.0260,0.72900,0.00424,0.1140,0.492
5,Gimmie Some Salt,0.712,0.6790,-6.518,0.0254,0.07910,0.68400,0.1120,0.552
6,Good Arms vs Bad Arms,0.363,0.8010,-5.860,0.0339,0.12200,0.00554,0.2800,0.211
7,Seville,0.626,0.6830,-14.811,0.0471,0.19400,0.91600,0.0939,0.858
8,In the Yard,0.548,0.4960,-10.762,0.0292,0.89400,0.41400,0.0925,0.249
9,Cato As A Pun,0.410,0.8460,-4.969,0.0420,0.00112,0.00468,0.3060,0.523


In [105]:
def calculate_euclidean_distance(v1, v2):
    return np.linalg.norm(v1 - v2)

def calculate_score(pd1, pd2):
    if pd1.shape[1] != pd2.shape[1]:
        raise ValueError("Dataframes must have the same number of features.")
    
    for _, x in pd1.iterrows():
        fx = np.array(x[1:].values)
        dist = list()
        for _, y in pd2.iterrows():
            fy = np.array(y[1:].values)
            dist.append(calculate_euclidean_distance(fx, fy))
        print(np.mean(dist))

In [106]:
calculate_score(our_recs, recs_from_spotify)

4.7089824932069755
4.74320481587444
4.767959191497498
4.751238434214102
4.738988098627883
4.707992417921841
4.752035254686284
4.7640775014676
4.745790203867066
4.7696688072375775
